In [1]:
import numpy as np
import pandas as pd
import requests
import lxml
from bs4 import BeautifulSoup

In [2]:
source=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [3]:
soup=BeautifulSoup(source,'lxml')

In [4]:
soup.find_all('td')
print('found all')

found all


In [5]:
cnt=0
dic={}
postcode=[]
borough=[]
neighbourhood=[]

In [6]:
for trow in soup.find_all("td"):
    try:
        if(cnt==0):
            pst_code=trow.text
            cnt=cnt+1
            if(pst_code[0]!='M'):
                break
        elif(cnt==1):
            bor=trow.text
            cnt=cnt+1
        elif(cnt==2):
            neigh=trow.text
            cnt=0
            postcode.append(pst_code)
            borough.append(bor)
            neighbourhood.append(neigh)
    
    except:
        pass

In [7]:
df=pd.DataFrame(columns=['post_code','borough','neighbourhood'])
df.head()

,post_code,borough,neighbourhood


In [8]:
li=[]
for i in (neighbourhood):
    i=i[0:len(i)-1]
    li.append(i)


In [9]:
df['post_code']=postcode
df['borough']=borough
df['neighbourhood']=li
df.shape[0]

289

In [10]:
condition=df['borough']!="Not assigned"
df=df[condition]

In [11]:
df.head()

,post_code,borough,neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [12]:
li1=[]
for row in df.itertuples():
    li=[]
    for j in df.itertuples():
        if (row.post_code==j.post_code):
            li.append(j.neighbourhood)
    li1.append(li)  

In [13]:
li2=[]
for i in li1:
    st=",".join(i)
    li2.append(st)


In [14]:
df=df.drop(columns=['neighbourhood'])

In [15]:
df['neighbouthood']=li2
df.head()

,post_code,borough,neighbouthood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
5,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M6A,North York,"Lawrence Heights,Lawrence Manor"


In [16]:
df.drop_duplicates(inplace=True)

In [17]:
df.head(10)

,post_code,borough,neighbouthood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M6A,North York,"Lawrence Heights,Lawrence Manor"
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Rouge,Malvern"
14,M3B,North York,Don Mills North
15,M4B,East York,"Woodbine Gardens,Parkview Hill"
17,M5B,Downtown Toronto,"Ryerson,Garden District"


In [18]:
df.loc[8,'post_code']='M7A'

In [19]:
df.head(10)

,post_code,borough,neighbouthood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M6A,North York,"Lawrence Heights,Lawrence Manor"
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Rouge,Malvern"
14,M3B,North York,Don Mills North
15,M4B,East York,"Woodbine Gardens,Parkview Hill"
17,M5B,Downtown Toronto,"Ryerson,Garden District"


In [20]:
df=df[:-1]
df.head()

,post_code,borough,neighbouthood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M6A,North York,"Lawrence Heights,Lawrence Manor"
8,M7A,Queen's Park,Not assigned


In [21]:
df.shape

(103, 3)

In [22]:
!pip install geocoder

In [23]:
df1=pd.read_csv('data.csv')
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
df=df.set_index('post_code')
df.head()

,borough,neighbouthood
post_code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Not assigned


In [25]:
df1=df1.set_index('Postal Code')
df1.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [26]:
df=df.join(df1)

In [27]:
df.head()

,borough,neighbouthood,Latitude,Longitude
post_code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
M7A,Queen's Park,Not assigned,43.662301,-79.389494


In [28]:
!pip install folium

In [29]:
import folium
map=folium.Map(location=[43.6532,-79.3832],zoom_start=10,tiles='Mapbox bright')

fg=folium.FeatureGroup(name="tornoto clusters")
for lat,lon,name in zip(df['Latitude'],df['Longitude'],df['borough']):
    fg.add_child(folium.Marker(location=[lat,lon],popup=(folium.Popup(name))))

map.add_child(fg)
map